In [2]:
import numpy as np
import pandas as pd
from ax.service.ax_client import AxClient
from ax.service.utils.instantiation import ObjectiveProperties
from tqdm import tqdm
import statistics
from pprint import pprint
from main import Game
from AI import MDP2

In [3]:
ax_client = AxClient(random_seed=42)


ax_client.create_experiment(
    name="moo_experiment1",
    parameters=[
        {
            "name": f"m1-3",
            "type": "range",
            "bounds": [250, 750],
            "value_type": "int"},
        {
            "name": f"m4-6",
            "type": "range",
            "bounds": [25, 100],
            "value_type": "int"},
        {
            "name": f"m7-9",
            "type": "range",
            "bounds": [8, 50],
            "value_type": "int"},
        {
            "name": f"m10-15",
            "type": "range",
            "bounds": [5, 20],
            "value_type": "int"},
        {
            "name": f"depth_scale",
            "type": "range",
            "bounds": [2, 9],
            "value_type": "float"},

    ],
    objectives={
        "score": ObjectiveProperties(minimize=False),
        "time_per_move": ObjectiveProperties(minimize=True)
    },
    parameter_constraints=["m1-3 >= m4-6", "m4-6 >= m7-9", "m7-9 >= m10-15"],
    overwrite_existing_experiment=False,
    is_test=False,
)

[INFO 07-11 22:24:44] ax.service.ax_client: Starting optimization with verbose logging. To disable logging, set the `verbose_logging` argument to `False`. Note that float values in the logs are rounded to 6 decimal points.
[WARNING 07-11 22:24:44] ax.service.ax_client: Random seed set to 42. Note that this setting only affects the Sobol quasi-random generator and BoTorch-powered Bayesian optimization models. For the latter models, setting random seed to the same number for two optimizations will make the generated trials similar, but not exactly the same, and over time the trials will diverge more.
[INFO 07-11 22:24:44] ax.service.utils.instantiation: Due to non-specification, we will use the heuristic for selecting objective thresholds.
[INFO 07-11 22:24:44] ax.service.utils.instantiation: Created search space: SearchSpace(parameters=[RangeParameter(name='m1-3', parameter_type=INT, range=[250, 750]), RangeParameter(name='m4-6', parameter_type=INT, range=[25, 100]), RangeParameter(name

In [4]:
df = pd.read_csv("saved games/MDP2.csv")
df["time_per_move"] = df["Duration"] / df["num_moves"]

df_renamed = df.rename(columns={'Score': 'score',
                                                'time_per_move': 'time_per_move',
                                                'core_param_0: (1-3)': 'm1-3',
                                                'core_param_1: (4-6)': 'm4-6',
                                                'core_param_2: (7-9)': 'm7-9',
                                                'core_param_3: (10-15)': 'm10-15',
                                                'core_param_4: 2/4 strength ratio': 'depth_scale'})
df_renamed = df_renamed.drop(["Board", "Model", "top_proportion", "num_moves", "Duration"], axis=1)

In [6]:
def evaluate(parameters: dict, total_runs: int = 10):
    parameters["depth_scale"] = round(parameters["depth_scale"], 2)
    param_array = np.array([
        parameters["m1-3"],
        parameters["m4-6"],
        parameters["m7-9"],
        parameters["m10-15"],
        parameters["depth_scale"],
    ])

    scores = []
    time_per_moves = []

    matching_rows = df_renamed.loc[
        (df_renamed['m1-3'] == parameters["m1-3"]) &
        (df_renamed['m4-6'] == parameters["m4-6"]) &
        (df_renamed['m7-9'] == parameters["m7-9"]) &
        (df_renamed['m10-15'] == parameters["m10-15"]) &
        (df_renamed['depth_scale'] == parameters["depth_scale"])
    ]

    if not matching_rows.empty:
        num_occurrences = len(matching_rows)
        total_runs -= num_occurrences
        scores.extend(matching_rows["score"].tolist())
        time_per_moves.extend(matching_rows["time_per_move"].tolist())

        print(f"{num_occurrences} with current parameters:\n"
              f"avg score = {matching_rows['score'].mean()}"
              f"avg time_per_move = {matching_rows['time_per_move'].mean()}\n Parameters:")
    else:
        print("No existing data for these parameters:")
    pprint(parameters)

    for game_run in range(total_runs):
        g = Game(use_gui=False, no_display=True)
        g.setup_board()
        m = MDP2(g, game_obj=Game, verbose=False, best_proportion=1, core_params=param_array)
        print(f"starting run {game_run} of {total_runs} with parameter array {param_array}")
        current_score, current_time_per_move = m.run()
        scores.append(current_score)
        time_per_moves.append(current_time_per_move)
    to_return = {"score": statistics.mean(scores),
                 "time_per_move": statistics.mean(time_per_moves)}

    return to_return

In [ ]:
for i in tqdm(range(30)):
    print(f"starting parameter iteration {i}")
    parameters, trial_index = ax_client.get_next_trial()
    ax_client.complete_trial(trial_index=trial_index, raw_data=evaluate(parameters))

  0%|          | 0/30 [00:00<?, ?it/s][INFO 07-11 22:20:20] ax.service.ax_client: Generated new trial 0 with parameters {'m1-3': 449, 'm4-6': 63, 'm7-9': 21, 'm10-15': 13, 'depth_scale': 5.327293}.


starting parameter iteration 0
No existing data for these parameters:
{'depth_scale': 5.32729332242161,
 'm1-3': 449,
 'm10-15': 13,
 'm4-6': 63,
 'm7-9': 21}
starting run 0 with parameter array [449.    63.    21.    13.     5.33]
--------------------SCORE: 0--------------------
0   0   0   0   
0   0   0   0   
0   0   0   0   
0   2   2   2   
